Importamos las librerias a utilizar

In [2]:
import pandas as pd
import numpy as np

Para poder usar Big Query (BQ) es necesario instalar las librerias de google. Para ello ejecutamos el siguiente código.

In [16]:
pip install --upgrade google-api-python-client

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install pandas-gbq

Ahora definimos la dirección de donde se obtendrán los datos y comprobamos que los datos se hayan cargado

In [3]:
url='https://www.cbinsights.com/research-unicorn-companies'
tabla= pd.read_html(url)[0]

In [5]:
df = pd.DataFrame(tabla)

Empezamos con el procesamiento de datos. Para este dataframe es necesario renombrar las etiquetas de las columnas, dado que la plataforma de google sólo acepta carateres alphanumericos guiones bajos y hasta 300 caracteres

In [17]:
df=df.rename(columns={'Valuation ($B)':'Valuation', 'Date Joined':'Date'})

In [41]:
df=df.rename(columns={'Select Investors':'Select_investor'})

In [25]:
df["Valuation"]=df["Valuation"].apply(lambda x: x.replace("$",""))

In [36]:
df['Date'] = pd.to_datetime(df['Date'], format="%m/%d/%Y")
df['Date'] =df['Date'].dt.strftime('%d/%m/%Y')
df['Valuation']=pd.to_numeric(df['Valuation'])

In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1183 entries, 0 to 1182
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Company          1183 non-null   object 
 1   Valuation        1183 non-null   float64
 2   Date             1183 non-null   object 
 3   Country          1183 non-null   object 
 4   City             1166 non-null   object 
 5   Industry         1183 non-null   object 
 6   Select_investor  1182 non-null   object 
dtypes: float64(1), object(6)
memory usage: 64.8+ KB


In [37]:
df.head()

,Company,Valuation,Date,Country,City,Industry,Select Investors
0,ByteDance,140.0,07/04/2017,China,Beijing,Artificial intelligence,"Sequoia Capital China, SIG Asia Investments, S..."
1,SpaceX,127.0,01/12/2012,United States,Hawthorne,Other,"Founders Fund, Draper Fisher Jurvetson, Rothen..."
2,SHEIN,100.0,03/07/2018,China,Shenzhen,E-commerce & direct-to-consumer,"Tiger Global Management, Sequoia Capital China..."
3,Stripe,95.0,23/01/2014,United States,San Francisco,Fintech,"Khosla Ventures, LowercaseCapital, capitalG"
4,Canva,40.0,08/01/2018,Australia,Surry Hills,Internet software & services,"Sequoia Capital China, Blackbird Ventures, Mat..."


Una vez procesada la información, cargamos las credenciales de la cuenta de servicio para cargar los datos a BQ

In [8]:
from google.oauth2 import service_account # para generar conexion con BigQuery
bq_cred = service_account.Credentials.from_service_account_file('C:/local/archivo.json')
bq_cred

Ahora sólo cargamos los datos. Hay que considerar que los valores de project_id y el nombre de la columna cambia con respecto a cada proyecto.

In [43]:
 # para interactuar con BQ
import pandas_gbq
pandas_gbq.to_gbq(df, 'nombre_conjunto_de_datos.nombre_tabla', project_id= 'id del proyecto', if_exists= 'replace', credentials = bq_cred)

100%|████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]


Y listo! terminamos con el procesamiento y carga de datos en BQ.